In [1]:
import sys 
sys.path.append('../../')
from config.HIVESERVER2_token import *

import pandas as pd
import pendulum
from tqdm import trange
from jinja2 import Template
from pyutil.pysql.sql import SQLClient

client = SQLClient('mixx')

q = '''
SELECT 
    a.*
    , b.feature AS role_feature
FROM (
    SELECT *
    FROM hive.ml_tmp_db.wgame_bunlde_sequence_sample 
    WHERE dt = '{ds}'
) as a
INNER JOIN (
    SELECT 
        IF(feature_group = 'common', CONCAT(role_id,acc_id,dt), CONCAT(acc_id,role_id,dt)) as id
        , CONCAT_WS(
            U&'\002',
            ARRAY_AGG(feature)
        ) AS feature
    FROM hive.ml_tmp_db.wgame_bundle_dynamic_pricing_feature_res_ratityv2
    WHERE feature_group in ('res_ratity','common') 
        AND dt = '{ds}'
    GROUP BY IF(feature_group = 'common', CONCAT(role_id,acc_id,dt), CONCAT(acc_id,role_id,dt))
) b
ON concat(a.open_id,a.user_id,a.dt) = b.id
'''

DS = pendulum.Date(2023,9,10)

client.reset()
for i in trange(60):
    ds = DS.add(days=-i).to_date_string()
    temp = Template(open("sql/common_feature.sql",'r').read())
    [client.execute_trino_query(q) for q in temp.render(feature_list = feature_name, ds = ds).split(';')]

client.reset()
for i in trange(60):
    ds = DS.add(days=-i).to_date_string()
    temp = Template(open("sql/res_feature.sql",'r').read())
    [client.execute_trino_query(q) for q in temp.render(ds = ds).split(';')]

client.reset()
temp = Template(open("sql/0.0.4.sql",'r').read())
for i in trange(30):
    ds = DS.add(days=-i).to_date_string()
    [client.execute_trino_query(q) for q in temp.render(ds = ds).split(';')]

In [20]:
# DS = pendulum.Date(2023,8,1)
q = "select * from hive.ml_tmp_db.wgame_bunlde_sequence_sample where dt = '{ds}' "
data = []
client.reset()
for i in trange(30):
    ds = DS.add(days=-i).to_date_string()
    client.execute_trino_query(q.format(ds = ds))
    data.append(client.to_pandas())

dataset = pd.concat(data, ignore_index=True)
dataset.to_parquet('../../data/raw/0.0.4.pqt', compression='gzip')

100%|██████████| 30/30 [07:21<00:00, 14.70s/it]


In [18]:
# 如果需要tfrecord格式的文件，进行下面的操作

from lmtk.feature_extraction import FeatureExtractor, FEATURE_BIT
from cityhash import CityHash32
import numpy as np


fe = FeatureExtractor('../../config/dynamic_pricing/0.0.5.conf')


d = (
    dataset
    # .head(10)
    .assign(
        sid = lambda x : (x.open_id + x.user_id + x.dt + x.trigger_type).apply(CityHash32),
        feature = lambda x: x.feature.apply(fe.extract)
    )
)

In [39]:
import tensorflow as tf
from tensorflow.train import BytesList, FloatList, Int64List, Example, Feature
from cityhash import CityHash64, CityHash32

def _extract(x):
    feature = (x['feature'] or '') + '\x02' + x['role_feature']
    feature = fe.extract(feature)
    
    example = Example()
    example.features.feature['sid'].CopyFrom(Feature(int64_list=Int64List(value=[CityHash32(x['open_id'] + x['user_id'] + x['dt'] + x['trigger_type'])])))
    example.features.feature['trigger_type'].CopyFrom(Feature(bytes_list=BytesList(value=[x['trigger_type'].encode('utf-8')])))
    example.features.feature['max7pay'].CopyFrom(Feature(bytes_list=BytesList(value=[x['max7pay'].encode('utf-8')])))
    for k, v in feature.items():
        if k == 'fids':
            example.features.feature[k].CopyFrom(Feature(int64_list=Int64List(value=v)))
        else:
            example.features.feature[k[2:]].CopyFrom(Feature(float_list=FloatList(value=v)))
    
    return example

In [42]:
from multiprocessing.dummy import Pool
pool = Pool(16)
features = pool.map(_extract, dataset.to_dict('records'))
pool.close()
pool.join()

In [43]:
with tf.io.TFRecordWriter('../../data/raw/0.0.4.tfrecord') as writer:
    for f in features:
        writer.write(f.SerializeToString())

In [11]:
# 累充
field = dict(
    #num
    field1 = [
        "acc_first_purchase_price_usd",
        "acc_total_charge_sum_usd",
        "acc_total_refund_sum_usd",
        "role_first_purchase_price_usd",  
        "role_last_purchase_price_usd",
    ],

    # cnt类特征
    # num
    field2 = [
        "acc_role_cnt",
        "acc_total_charge_cnt",
        "acc_total_refund_cnt",
        "role_total_charge_cnt",
        "role_total_refund_cnt",
    ],

    # num
    field3 = [
        "bundle_pop_buy_count_last_day",
        "bundle_pop_buy_count_last_day_campaign_stage_trigger",
        "bundle_pop_buy_count_last_day_tech_unlock_trigger",
        "bundle_pop_buy_count_last_week",
        "bundle_pop_buy_count_last_week_adjust",
        
        "bundle_pop_count_campaign_stage_trigger_9999",
        "bundle_pop_count_last_day",
        "bundle_pop_count_last_day_campaign_stage_trigger",
        "bundle_pop_count_last_day_tech_unlock_trigger",
        "bundle_pop_count_last_week",
        "bundle_pop_count_last_week_adjust",
        "bundle_pop_count_tech_unlock_trigger_9999",
        
        
    ],

    # cat
    field4 = [
        "bundle_pop_buy_max_last_day",
        "bundle_pop_buy_max_last_week",
        "bundle_pop_buy_max_last_week_adjust",
        "bundle_pop_buy_max_price_campaign_stage_trigger",
        "bundle_pop_buy_max_price_campaign_stage_trigger_commax",
        "bundle_pop_buy_max_price_tech_unlock_trigger",
        "bundle_pop_buy_max_price_tech_unlock_trigger_commax",
        
        "purchase_price_max",
        "purchase_price_max_last_3days",
        "purchase_price_max_last_day",
        "purchase_price_max_last_month",
        "purchase_price_max_last_week",
        "purchase_price_max_last_week_adjust",
    ],

    # num + log cat
    field5 = [
        "bundle_pop_buy_sum_last_day",
        "bundle_pop_buy_sum_last_week",
        "bundle_pop_buy_sum_last_week_adjust",
        
        "purchase_price_1499_last_week",
        "purchase_price_1499_last_week_adjust",
        "purchase_price_1999_last_week",
        "purchase_price_1999_last_week_adjust",
        "purchase_price_2999_last_week",
        "purchase_price_2999_last_week_adjust",
        "purchase_price_4999_last_week",
        "purchase_price_4999_last_week_adjust",
        "purchase_price_499_last_week",
        "purchase_price_499_last_week_adjust",
        "purchase_price_9999_last_week",
        "purchase_price_9999_last_week_adjust",
        "purchase_price_999_last_week",
        "purchase_price_999_last_week_adjust",
        
        "purchase_price_sum_last_3days",
        "purchase_price_sum_last_day",
        "purchase_price_sum_last_month",
        "purchase_price_sum_last_week",
        "purchase_price_sum_last_week_adjust",
        
        "role_total_charge_sum_usd",
        "role_total_refund_sum_usd",
    ],

    # num + logcat
    field6 = [
        "money_flow_in_cout_last_day",
        "money_flow_in_cout_last_week",
        "money_flow_in_cout_last_week_adjust",
        "money_flow_out_cout_last_day",
        "money_flow_out_cout_last_week",
        "money_flow_out_cout_last_week_adjust",
    ],

    # other
    field7 = [
        "level",
        "max_role_level",
        "role_last_login_duration",
        "role_lifespan",
        "role_login_last_now",
        "role_login_secondlast_last",
        "role_login_secondlast_now",
        "role_logout_last_now",
        "role_no",
        "role_purchase_first_last",
        "role_purchase_first_now",
        "role_purchase_last_now",
        "vip_level",
        
        "purchase_price_sum_max_ratio",
        "purchase_price_sum_max_ratio_com2",
    ],

    field8 = common_feature_boolean,
    field9 = common_feature_ratio,
    field10 = diamond_remain_feature,
    field11 = cat_feature,
    field12 = [f'last{i}bundle' for i in range(1,6)]
)

In [4]:
cat_feature = [
    'svr_region'
]

common_feature_boolean = [
    "role_create_os_android",
    "role_create_os_ios",
    "if_accounts_first_role",
    "if_pruchase_active_in_1day",
    "if_pruchase_active_in_30day",
    "if_pruchase_active_in_3day",
    "if_pruchase_active_in_7day",
    "is_acc_transfer_from_old",
    "is_acc_transfer_to_new",
    "is_role_transfer_from_old",
    "is_role_transfer_to_new",
    "is_valid",
    "role_type_fraud",
    "role_type_no_install_info",
]

common_feature_numeric = [
    # "acc_first_purchase_price_usd",
    # "acc_role_cnt",
    # "acc_total_charge_cnt",
    # "acc_total_charge_sum_usd",
    # "acc_total_refund_cnt",
    # "acc_total_refund_sum_usd",
    # "bundle_pop_buy_count_last_day",
    # "bundle_pop_buy_count_last_day_campaign_stage_trigger",
    # "bundle_pop_buy_count_last_day_tech_unlock_trigger",
    # "bundle_pop_buy_count_last_week",
    # "bundle_pop_buy_count_last_week_adjust",
    # "bundle_pop_buy_max_last_day",
    # "bundle_pop_buy_max_last_week",
    # "bundle_pop_buy_max_last_week_adjust",
    # "bundle_pop_buy_max_price_campaign_stage_trigger",
    # "bundle_pop_buy_max_price_campaign_stage_trigger_commax",
    # "bundle_pop_buy_max_price_tech_unlock_trigger",
    # "bundle_pop_buy_max_price_tech_unlock_trigger_commax",
    # "bundle_pop_buy_sum_last_day",
    # "bundle_pop_buy_sum_last_week",
    # "bundle_pop_buy_sum_last_week_adjust",
    # "bundle_pop_count_campaign_stage_trigger_9999",
    # "bundle_pop_count_last_day",
    # "bundle_pop_count_last_day_campaign_stage_trigger",
    # "bundle_pop_count_last_day_tech_unlock_trigger",
    # "bundle_pop_count_last_week",
    # "bundle_pop_count_last_week_adjust",
    # "bundle_pop_count_tech_unlock_trigger_9999",

    # "level",
    # "max_role_level",
    # "money_flow_in_cout_last_day",
    # "money_flow_in_cout_last_week",
    # "money_flow_in_cout_last_week_adjust",
    # "money_flow_out_cout_last_day",
    # "money_flow_out_cout_last_week",
    # "money_flow_out_cout_last_week_adjust",
    # "purchase_price_1499_last_week",
    # "purchase_price_1499_last_week_adjust",
    # "purchase_price_1999_last_week",
    # "purchase_price_1999_last_week_adjust",
    # "purchase_price_2999_last_week",
    # "purchase_price_2999_last_week_adjust",
    # "purchase_price_4999_last_week",
    # "purchase_price_4999_last_week_adjust",
    # "purchase_price_499_last_week",
    # "purchase_price_499_last_week_adjust",
    # "purchase_price_9999_last_week",
    # "purchase_price_9999_last_week_adjust",
    # "purchase_price_999_last_week",
    # "purchase_price_999_last_week_adjust",
    # "purchase_price_max",
    # "purchase_price_max_last_3days",
    # "purchase_price_max_last_day",
    # "purchase_price_max_last_month",
    # "purchase_price_max_last_week",
    # "purchase_price_max_last_week_adjust",
    # "purchase_price_sum_last_3days",
    # "purchase_price_sum_last_day",
    # "purchase_price_sum_last_month",
    # "purchase_price_sum_last_week",
    # "purchase_price_sum_last_week_adjust",

    
    # "role_first_purchase_price_usd",
    # "role_last_login_duration",
    # "role_last_purchase_price_usd",
    # "role_lifespan",
    # "role_login_last_now",
    # "role_login_secondlast_last",
    # "role_login_secondlast_now",
    # "role_logout_last_now",
    # "role_no",
    # "role_purchase_first_last",
    # "role_purchase_first_now",
    # "role_purchase_last_now",
    # "role_total_charge_cnt",
    # "role_total_charge_sum_usd",
    # "role_total_refund_cnt",
    # "role_total_refund_sum_usd",
    
    # "vip_level",
    
    # "purchase_price_sum_max_ratio",
    # "purchase_price_sum_max_ratio_com2",
]

common_feature_ratio = [
    'resource_rarity',
    'speedup_rarity',
    'top_key_rarity',
    'officer_bar_ticket_rarity',
    'common_equipment_exp_consume_levelup_1_diff_num_relatively',
    'common_equipment_exp_consume_blueprint_1_diff_num_relatively',
    'common_equipment_exp_rarity',
    'airforce_equipment_exp_consume_levelup_1_diff_num_relatively',
    'airforce_equipment_exp_consume_blueprint_1_diff_num_relatively',
    'airforce_equipment_exp_rarity',
    'ammunition_consume_levelup_1_diff_num_relatively',
    'ammunition_consume_levelup_10_diff_num_relatively',
    'ammunition_consume_blueprint_1_diff_num_relatively',
    'ammunition_consume_blueprint_10_diff_num_relatively',
    'ammunition_rarity',
    'airforce_ammunition_consume_levelup_1_diff_num_relatively',
    'airforce_ammunition_consume_levelup_10_diff_num_relatively',
    'airforce_ammunition_consume_blueprint_1_diff_num_relatively',
    'airforce_ammunition_consume_blueprint_10_diff_num_relatively',
    'airforce_ammunition_rarity',
    'elements_consume_blueprint_1_diff_num_relatively',
    'elements_consume_blueprint_10_diff_num_relatively',
    'elements_consume_reform_diff_num_relatively',
    'elements_rarity',
    'airforce_elements_consume_blueprint_1_diff_num_relatively',
    'airforce_elements_consume_blueprint_10_diff_num_relatively',
    'airforce_elements_consume_reform_diff_num_relatively',
    'airforce_elements_rarity',
    'replacement_wrench_consume_mechanic_1_diff_num_relatively',
    'replacement_wrench_rarity',
    'officer_exp_consume_lvlup_1_diff_num_relatively',
    'officer_exp_rarity',
    'component_consume_diff_num_relatively',
    'chest_choice_replacement_rarity',

    "buy_rate_last_day",
    "buy_rate_last_day_campaign_stage_trigger",
    "buy_rate_last_day_tech_unlock_trigger",
    "buy_rate_last_week",
]


item_remain_feature = [
    "item_remain_usdlg2",
]

diamond_remain_feature = [
    'diamond_paid_remain_rmb',
    'diamond_paid_remain_usd'
]

In [52]:
d1 = dataset.reset_index().assign(
    role_feature = lambda x : x.role_feature.apply(lambda x: '\x02'.join(eval(x))),
    feature = lambda x: x.feature.apply(lambda x: x + '\x02' if x is not None else '') + x.role_feature
).filter(items = ['feature', 'trigger_type', 'max7pay'])

In [52]:
from cityhash import CityHash32

price_chain = ['499','999','1499', '1999', '2999','4999', '9999']
p2idx = {p:i for i,p in enumerate(price_chain)}
idx2p = {i:p for i,p in enumerate(price_chain)}

def extract(x):
    if x is not None:
        x = [v.split('\x03') for v in x.split('\x02')]
        x = {
            v[0] : v[1].split('\x04')
            for v in x
        }
    else:
        x = {}

    return x

d1 = dataset.assign(
    sid = lambda x: (x.open_id + x.user_id + x.dt + x.trigger_type).apply(CityHash32),
    max7pay = lambda x: x.max7pay.apply(lambda x : p2idx.get(x)),
    role_feature = lambda x : x.role_feature.apply(eval)
).assign(
    bundle_feature = lambda x: x.feature.apply(extract),
    role_feature = lambda x: x.role_feature.apply(lambda v: [extract(_) for _ in v])
)